In [1]:
import time
import os 
import numpy as np
from scipy import sparse
import pandas as pd
import h5py
import anndata


from dredFISH.Analysis import basicu

In [2]:
ddir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/'
res_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/res_dpnmf/'

In [3]:
%%time
# read in data
f = os.path.join(ddir, 'scrna_ss_ctxhippo_a_exon_count_matrix_v5.h5ad')
adata = anndata.read(f,
                     # backed='r',
                    )
adata

CPU times: user 1.36 s, sys: 3.98 s, total: 5.34 s
Wall time: 5.34 s


AnnData object with n_obs × n_vars = 73347 × 45768
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [4]:
# get the NN genes
f = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/10X/genes.npy' 
tenx_genes = np.load(f)
f = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/SmartSeq/genes.npy' 
smrt_genes = np.load(f)
assert np.all(tenx_genes == smrt_genes)
nn_genes = tenx_genes
nn_genes.shape

(10861,)

# prep
- select relevant genes only
- subsample cells
- dense matrix
- numpy format


In [5]:
nn_genes_idx = basicu.get_index_from_array(adata.var.index.values, nn_genes)
(nn_genes_idx == -1).sum()

0

In [6]:
%%time
adata_nng = adata[:,adata.var.iloc[nn_genes_idx].index].copy()
adata_nng

CPU times: user 9.74 s, sys: 1.91 s, total: 11.7 s
Wall time: 11.7 s


AnnData object with n_obs × n_vars = 73347 × 10861
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [7]:
# all cells
# randomly select cells L5 (allow oversample)

# randomly select cells L3 (does not allow oversamp)
clstcol = 'subclass_label'
n = 100
dfsub, idxsub = basicu.stratified_sample(adata_nng.obs, clstcol, n, 
                                         return_idx=True, group_keys=False, sort=False, 
                                         random_state=0, 
                                        )

adatasub = adata_nng[adata_nng.obs.iloc[idxsub].index].copy()
adatasub

AnnData object with n_obs × n_vars = 3926 × 10861
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'cel

In [8]:
adatasub.obs.groupby(clstcol).size()

subclass_label
Astro              100
CA1-ProS           100
CA2-IG-FC           19
CA3                100
CR                  38
CT SUB             100
Car3               100
DG                 100
Endo               100
L2 IT ENTl         100
L2 IT ENTm          42
L2/3 IT CTX        100
L2/3 IT ENTl       100
L2/3 IT PPP        100
L2/3 IT RHP        100
L3 IT ENT          100
L4 RSP-ACA         100
L4/5 IT CTX        100
L5 IT CTX          100
L5 PPP              49
L5 PT CTX          100
L5/6 IT TPE-ENT    100
L5/6 NP CTX        100
L6 CT CTX          100
L6 IT CTX          100
L6 IT ENTl          78
L6b CTX            100
L6b/CT ENT         100
Lamp5              100
Meis2              100
Micro-PVM          100
NP PPP             100
NP SUB             100
Oligo              100
Pvalb              100
SMC-Peri           100
SUB-ProS           100
Sncg               100
Sst                100
Sst Chodl          100
VLMC               100
Vip                100
dtype: int64

In [9]:
# convert to dense and format it; need the matrix and the labels
X = adatasub.X.toarray().T # gene by cell
y = adatasub.obs[clstcol].values
X.shape, y.shape
y

['L5 PT CTX', 'L5 PT CTX', 'L5 PT CTX', 'L5 PT CTX', 'L5 PT CTX', ..., 'NP SUB', 'NP SUB', 'NP SUB', 'NP SUB', 'NP SUB']
Length: 3926
Categories (42, object): ['Astro', 'CA1-ProS', 'CA2-IG-FC', 'CA3', ..., 'Sst', 'Sst Chodl', 'VLMC', 'Vip']

In [10]:
out_X = os.path.join(res_dir, 'test_X.npy') 
out_y = os.path.join(res_dir, 'test_y.npy') 
np.save(out_X, X)
np.save(out_y, y)

In [11]:
print(out_X)

/bigstore/GeneralStorage/fangming/projects/dredfish/res_dpnmf/test_X.npy
